## Свой numpy

код написан с рекомендациями pep

In [5]:
import numpy as np

In [6]:
# pip install pycodestyle flake8 pycodestyle_magic

In [7]:
%load_ext pycodestyle_magic

In [8]:
%%pycodestyle

a = 1 '# eee'
s = 2

In [9]:
float('-inf')

-inf

In [10]:
class Null(float):
    def __repr__(self):
        return 'Null'
    def __add__(self, other):
        return Null()
    def __radd__(self, other):
        return Null()
    def __sub__(self, other):
        return Null()
    def __rsub__(self, other):
        return Null()
    def __mul__(self, other):
        return Null()
    def __rmul__(self, other):
        return Null()
    def __truediv__(self, other):
        return Null()
    def __rtruediv__(self, other):
        return Null()
    def __eq__(self, other):
        return False
    def __ne__(self, other):
        return True
    def __lt__(self, other):
        return False
    def __le__(self, other):
        return False
    def __gt__(self, other):
        return False
    def __ge__(self, other):
        return False

In [11]:
(np.nan != 2) == (Null() != 2)

True

In [12]:
(np.nan == np.nan) == (Null() == Null())

True

In [13]:
(np.nan != np.nan) == (Null() != Null())

True

In [14]:
# %%pycodestyle

class Array(list):
    
    def __repr__(self):
        return super().__repr__()

    """Сложение, когда аргумент слева"""
    def __add__(self, other):
        if isinstance(other, (int, float)):
            return Array([x + other for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Сложение векторов разной длины недопустимо')
            else:
                return Array([x + y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя сложить с {other}")
    """Сложение, когда аргумент справа"""
    def __radd__(self, other):
        return self + other

    """Вычитание, когда аргумент слева"""
    def __sub__(self, other):
        return self.sub(other, 'left')
    """Вычитание, когда аргумент справа"""
    def __rsub__(self, other):
        return self.sub(other, 'right')

    def sub(self, other, flag):
        if isinstance(other, (int, float)):
            if flag == 'left':
                return Array([x - other for x in self])
            elif flag == 'right':
                return Array([other - x for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Вычитание векторов разной длины недопустимо')
            else:
                return Array([x - y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя вычисть с {other}")

    """Умножение, когда аргумент слева"""
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            return Array([x * other for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Умножение векторов разной длины недопустимо')
            else:
                return Array([x * y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя умножить с {other}")
    """Умножение, когда аргумент справа"""
    def __rmul__(self, other):
        return self * other

    """Деление, когда аргумент слева"""
    def __truediv__(self, other):
        return self.truediv(other, 'left')
    """Деление, когда аргумент справа"""
    def __rtruediv__(self, other):
        return self.truediv(other, 'right')

    def truediv(self, other, flag):
        if isinstance(other, (int, float)):
            if flag == 'left':
                return Array([x / other for x in self])
            elif flag == 'right':
                return Array([other / x for x in self])
        elif isinstance(other, (Array, list, tuple)):
            if len(self) != len(other):
                raise ValueError('Деление векторов разной длины недопустимо')
            else:
                return Array([x / y for x, y in zip(self, other)])
        else:
            raise TypeError(f"Вектор нельзя разделить с {other}")
            
    """Добавление функционала слайсинга"""
    def __getitem__(self, item):
        if type(item) != tuple:
            item = super().__getitem__(item)
            if isinstance(item, (float, int, str)) or item == None :
                return item
            return Array(item)
        else:
            if type(item[0]) == slice:
                return Array([i[item[1]] for i in self[item[0]]])
            else:
                while len(item) > 0:
                    if isinstance(self, (int, float)) or item == None :
                        raise IndexError('Слишком много индексов для массива этого пространства')
                    self = self[item[0]]
                    item = item[1:]
                return self
    """Добавление функционала итерирования в цикле"""
    def __iter__(self):
        self.index = 0
        return self
    
    def __next__(self):
        if self.index >= len(self):
            raise StopIteration
        letter = self[self.index]
        self.index += 1 
        return letter

    def __eq__(self, other):
        if isinstance(other, (list, Array, tuple)) and len(self) == len(other):
            mas = []
            for i, j in zip(self, other):
                mas += [i == j]
            return Array(mas)
        elif isinstance(other, (int, float, Null, str)):
            mas = []
            for i in self:
                mas += [i == other]
            return Array(mas)
        else:
            return super().__eq__(other)
                

    """Находит min число в Array"""
    def min(self):
        mins = float('inf')
        for i in self:
            if not isinstance(i, (int, float)):
                i = Array(i).min()
            if i < mins:
                mins = i
        return mins
    """Находит max число в Array"""
    def max(self):
        maxs = float('-inf')
        for i in self:
            if not isinstance(i, (int, float)):
                i = Array(i).max()
            if i > maxs:
                maxs = i
        return maxs
            
                
    """Считает сумму всех чисел в Array"""
    def sum(self):
        sums = 0
        for i in self:
            if isinstance(i, (int, float)):
                sums += i
            else:
                sums += Array(i).sum()
        return sums
    """Считает кол-во всех элементов в Array"""
    def count_el(self):
        counts = 0
        for i in self:
            if isinstance(i, (int, float)):
                counts += 1
            else:
                counts += Array(i).count_el()
        return counts
    """Считает сумму и кол-во всех элементов в Array"""
    def sum_count(self):
        counts = 0
        sums = 0
        for i in self:
            if isinstance(i, (int, float)):
                counts += 1
                sums += i
            else:
                s, c = Array(i).sum_count()
                sums += s
                counts += c
                
        return sums, counts
    """Считает среднее значение элементов в Array"""
    def mean(self):
        s, c = self.sum_count()
        return s / c
    
    def std(self):
        return (((self - self.mean()) * (self - self.mean())).sum() / self.count_el() )** 0.5
    
    """в оригинальном np его нет, но что бы работал pandas нужен"""
    def dropna(self):
        mas = []
        for i in self:
            if not isinstance(i, Null) and i != None:
                mas += [i]
        return Array(mas)
    
    """округляет все числа в массиве"""
    def round(self, lens = 1):
        mas = []
        for i in range(len(self)):
            mas.append(round(self[i], lens))
        return mas

In [15]:
a = Array([1.43,3.23232,1212.323232])
a.round(3)

[1.43, 3.232, 1212.323]

## Тестирование базовых методов

In [16]:
list1 = [1,4,21,2]
a = Array(list1)
b = list1

In [17]:
sorted(b) == sorted(a)

True

In [18]:
b.append(1)
a.append(1)
b == a

[True, True, True, True, True]

In [19]:
a = Array([0, 10, 'bmv', 2000])

In [20]:
a.append(1)
a

[0, 10, 'bmv', 2000, 1]

## Тестирование знака равно

In [21]:
list1 = [1,2,3]
a = Array(list1)
b = np.array(list1)

In [22]:
(a == 2) == (b == 2)

array([ True,  True,  True])

In [23]:
a == [1,2], b == [1,2]

<ipython-input-23-9788d2e57f58>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a == [1,2], b == [1,2]


(False, False)

In [24]:
a == [1,2,3], b == [1,2,3]

([True, True, True], array([ True,  True,  True]))

In [25]:
a == [1,2,3, 4], b == [1,2,3, 4]

<ipython-input-25-b73ba61464d5>:1: DeprecationWarning: elementwise comparison failed; this will raise an error in the future.
  a == [1,2,3, 4], b == [1,2,3, 4]


(False, False)

## Тестирование цикла

In [26]:
list1 = [[1,2,3], [4,5,6], [7,8,9]]
a = Array(list1)
b = np.array(list1)

In [27]:
cool = True
for i, j in zip(b, a):
    if all(i == j):
        print('Ok')
    else:
        cool = False
        print('Ko')
print('--------------------')
if cool:
    print('Ошибок нет')
else:
    print('Ошибка!!!!!')

Ok
Ok
Ok
--------------------
Ошибок нет


# ошибка если 
#### 1. a = Array(1)
#### 2. list1 = [[[-4.0, -3.0, -2.0], [-1.0, 0.0, 1.0], [2.0, 3.0, 4.0]]  a = Array(list1) a * a
#### 3. Добавить возведение масива в степень!




In [28]:
list1 = [[[-4.0, -3.0, -2.0], [-1.0, 0.0, 1.0], [2.0, 3.0, 4.0]]]
a = Array(list1)
a * a

[]

## Тестирование dropna

In [29]:
l1 = [7,8,Null(), None]
my_arr = Array(l1)
my_arr.dropna() == [7,8]

[True, True]

## Тестирование mean, max, count...

In [30]:
l1 = [[[1,2,3], [4,5,6], [7,8,9]]]
l2 = [[[7,8,9], [1,2,3], [4,5,6]]]
l3 = [1,2,3]
l4 = [1]
l5 = [[[-4.0, -3.0, -2.0], [-1.0, 0.0, 1.0], [2.0, 3.0, 4.0]]]
tests = [l1,l2,l3,l4,l5]
cool = True
for test in tests:
    my_arr = Array(test)
    np_arr = np.array(test)
    if my_arr.min() != np_arr.min():
        cool = False
        print('Ko - min')
        
    if my_arr.max() != np_arr.max():
        cool = False
        print('Ko - max')
        
    if my_arr.mean() != np_arr.mean():
        cool = False
        print('Ko - mean')
        
    if my_arr.sum() != np_arr.sum():
        cool = False
        print('Ko - sum')
        
    if my_arr.std() != np_arr.std():
        cool = False
        print('Ko - std')
print('--------------------')
if cool:
    print('Ошибок нет')
else:
    print('Ошибка!!!!!')

--------------------
Ошибок нет


## Тестирование арифметики

In [31]:
def add(x, y):
    return x + y

def sub(x, y):
    return x - y

def mul(x, y):
    return x * y

def div(x, y):
    return x / y

In [32]:
a = Array

In [33]:
list1 = [1,2,3]
list2 = [4,5,6]
a = Array(list1)
a2 = np.array(list1)

b = Array(list2)
b2 = np.array(list2)

cool = True
for en, fun in enumerate([add, sub, mul, div]):
    if all(fun(a, b) == fun(a2, b2)): 
        print(f"{en} Ok - a {str(fun)[10:13]} b")
    else:
        cool = False
        print(f"{en} Ko - a {str(fun)[10:13]} b")
        
    if all(fun(b, a) == fun(b2, a2)):
        print(f"{en} Ok - b {str(fun)[10:13]} a\n")
    else:
        cool = False
        print(f"{en } Ko - b {str(fun)[10:13]} a\n")
        
    if all(fun(a, 3) == fun(a2, 3)):
        print(f"{en} Ok - a {str(fun)[10:13]} 3")
    else:
        cool = False
        print(f"{en} Ko - a {str(fun)[10:13]} 3")
        
    if all(fun(3, a) == fun(3, a2)):
        print(f"{en} Ok - 3 {str(fun)[10:13]} a\n")
    else:
        cool = False
        print(f"{en } Ko - 3 {str(fun)[10:13]} a\n")
    
print('--------------------')
if cool:
    print('Ошибок нет')
else:
    print('Ошибка!!!!!')

0 Ok - a add b
0 Ok - b add a

0 Ok - a add 3
0 Ok - 3 add a

1 Ok - a sub b
1 Ok - b sub a

1 Ok - a sub 3
1 Ok - 3 sub a

2 Ok - a mul b
2 Ok - b mul a

2 Ok - a mul 3
2 Ok - 3 mul a

3 Ok - a div b
3 Ok - b div a

3 Ok - a div 3
3 Ok - 3 div a

--------------------
Ошибок нет


## Тестирование слайсинга 2

In [34]:
def slice_intepritator(slices):
    if type(slices) == slice:
        if slices.start == slices.stop == slices.step == None:
            return ':'
        elif slices.start == None and slices.stop != None:
            return f' : {slices.stop}'
        elif slices.start != None and slices.stop == None:
            return f'{slices.start} : '
        else:
            return f'{slices.start} : {slices.stop}'
    else:
        return slices

In [35]:
# list1 = [[1,2,3], [4,5,6], [7,8,9], ['sds', 'dssc', 'cscs']]
list1 = [[1,2,3], [4,5,6], [7,8,9]]
a = Array(list1)
b = np.array(list1)
c = list1

In [36]:
slicesings = [[slice(0, 2), 1], [1, slice(0, 2)], [0, 1], [1, 0], [1, slice(None)],[slice(None), 1],\
              [1,slice(1, None)],[slice(1, None), 1]]
cool = True
for en, sl in enumerate(slicesings):
    print(f'array[{slice_intepritator(sl[0])}, {slice_intepritator(sl[1])}]', 'np:', b[sl[0], sl[1]], 'my:', a[sl[0], sl[1]])
    if np.all(b[sl[0], sl[1]] == a[sl[0], sl[1]]):
        print(en + 1, 'Ok')
    else:
        cool = False
        print(en + 1, 'Ko', [sl[0], sl[1]])
        
    print(f'array[{slice_intepritator(sl[0])}][{slice_intepritator(sl[1])}]','np:', b[sl[0]][sl[1]], 'my:', a[sl[0]][sl[1]])
    
    if np.all(b[sl[0]][sl[1]] == a[sl[0]][sl[1]]):
        print(en + 1, 'Ok', '\n')
    else:
        cool = False
        print(en + 1, 'Ko', [sl[0]], [sl[1]], '\n')
print('--------------------')
if cool:
    print('Ошибок нет')
else:
    print('Ошибка!!!!!')

array[0 : 2, 1] np: [2 5] my: [2, 5]
1 Ok
array[0 : 2][1] np: [4 5 6] my: [4, 5, 6]
1 Ok 

array[1, 0 : 2] np: [4 5] my: [4, 5]
2 Ok
array[1][0 : 2] np: [4 5] my: [4, 5]
2 Ok 

array[0, 1] np: 2 my: 2
3 Ok
array[0][1] np: 2 my: 2
3 Ok 

array[1, 0] np: 4 my: 4
4 Ok
array[1][0] np: 4 my: 4
4 Ok 

array[1, :] np: [4 5 6] my: [4, 5, 6]
5 Ok
array[1][:] np: [4 5 6] my: [4, 5, 6]
5 Ok 

array[:, 1] np: [2 5 8] my: [2, 5, 8]
6 Ok
array[:][1] np: [4 5 6] my: [4, 5, 6]
6 Ok 

array[1, 1 : ] np: [5 6] my: [5, 6]
7 Ok
array[1][1 : ] np: [5 6] my: [5, 6]
7 Ok 

array[1 : , 1] np: [5 8] my: [5, 8]
8 Ok
array[1 : ][1] np: [7 8 9] my: [7, 8, 9]
8 Ok 

--------------------
Ошибок нет


## Тестирование ошибок

In [37]:
a = Array([1,2,3])
b = Array([4,5,6,7])
print(a + b)

ValueError: Сложение векторов разной длины недопустимо

In [38]:
print(a + dict)

TypeError: Вектор нельзя сложить с <class 'dict'>

In [39]:
print(a * dict)

TypeError: Вектор нельзя умножить с <class 'dict'>

In [127]:
n = np.array([1,2,3])

In [128]:
a == n

array([ True,  True,  True])

In [129]:
np.array([1,2,3]) * np.array([1,2,3, 4])

ValueError: operands could not be broadcast together with shapes (3,) (4,) 

In [130]:
np.array([1,2,3]) * dict

TypeError: unsupported operand type(s) for *: 'int' and 'type'

In [131]:
list1 = [[1,2,3], [4,5,6], [7,8,9]]
a = Array(list1)
b = np.array(list1)

In [132]:
a[1,1,1]

IndexError: Слишком много индексов для массива этого пространства

In [133]:
b[1,1,1]

IndexError: too many indices for array: array is 2-dimensional, but 3 were indexed